In [1]:
# Import Necessary Libraries

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, Model, backend as K

2025-12-31 05:37:20.803821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767159440.998934      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767159441.057161      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Data Loading

DATA_PATH = "/kaggle/input/brats2020-training-data/BraTS2020_training_data/content/data"
MAX_FILES = 2000

def safe_load_h5(path):
    try:
        with h5py.File(path, 'r') as f:
            img = np.array(f['image']).astype(np.float32)
            msk = np.array(f['mask']).astype(np.float32)
            return img, msk
    except Exception as e:
        return None, None

print("Loading dataset...")
files = sorted([f for f in os.listdir(DATA_PATH) if f.endswith('.h5')])[:MAX_FILES]

images = []
masks = []

for fname in tqdm(files, desc="Loading files"):
    p = os.path.join(DATA_PATH, fname)
    img, msk = safe_load_h5(p)
    if img is None:
        continue
        
    if img.ndim == 2:
        img = np.expand_dims(img, -1)
    if msk.ndim == 2:
        msk = np.expand_dims(msk, -1)

    # Normalize image
    p99 = np.percentile(img, 99)
    img_norm = img / p99 if p99 > 0 else img
    img_norm = np.clip(img_norm, 0.0, 1.0)

    # Mask processing
    msk = np.clip(msk, 0.0, 1.0)

    images.append(img_norm)
    masks.append(msk)

X = np.array(images, dtype=np.float32)
Y = np.array(masks, dtype=np.float32)
print(f"Loaded shapes -> X: {X.shape}, Y: {Y.shape}")

Loading dataset...


Loading files: 100%|██████████| 2000/2000 [00:44<00:00, 45.33it/s]


Loaded shapes -> X: (2000, 240, 240, 4), Y: (2000, 240, 240, 3)


In [3]:
# Loss Functions

def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    bce = K.mean(bce)
    d_loss = dice_loss(y_true, y_pred)
    return bce + d_loss

In [4]:
# 3-Channel U-Net

def build_simple_multi_unet(input_shape=(240, 240, 4), num_classes=3):
    """U-Net that works well"""
    inputs = tf.keras.Input(shape=input_shape)
    
    # Downsample
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D(2)(c1)
    
    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D(2)(c2)
    
    # Bottleneck
    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, 3, activation='relu', padding='same')(c3)
    
    # Upsample
    u1 = layers.UpSampling2D(2)(c3)
    u1 = layers.concatenate([u1, c2])
    c4 = layers.Conv2D(64, 3, activation='relu', padding='same')(u1)
    c4 = layers.Conv2D(64, 3, activation='relu', padding='same')(c4)
    
    u2 = layers.UpSampling2D(2)(c4)
    u2 = layers.concatenate([u2, c1])
    c5 = layers.Conv2D(32, 3, activation='relu', padding='same')(u2)
    c5 = layers.Conv2D(32, 3, activation='relu', padding='same')(c5)
    
    # 3 output channels with sigmoid (independent segmentation)
    outputs = layers.Conv2D(num_classes, 1, activation='sigmoid')(c5)
    
    model = tf.keras.Model(inputs, outputs)
    return model

In [5]:
# Train/Val Split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.18, random_state=42)
print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Val: {X_val.shape}, {y_val.shape}")

Train: (1640, 240, 240, 4), (1640, 240, 240, 3)
Val: (360, 240, 240, 4), (360, 240, 240, 3)


In [6]:
# Build & Compile Model

print("Building 3-channel U-Net...")
model = build_simple_multi_unet(input_shape=X_train.shape[1:], num_classes=3)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=combined_loss,
    metrics=['accuracy', dice_coef]
)

model.summary()

Building 3-channel U-Net...


I0000 00:00:1767159501.380396      38 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 240, 240,  │      1,184 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 240, 240,  │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 120, 120,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 120, 120,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 120, 120,  │     36,928 │ conv2d_2[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 60, 60,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 60, 60,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 60, 60,    │    147,584 │ conv2d_4[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 120, 120,  │          0 │ conv2d_5[0][0]    │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 120, 120,  │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 192)              │            │ conv2d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 120, 120,  │    110,656 │ concatenate[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 120, 120,  │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_1     │ (None, 240, 240,  │          0 │ conv2d_7[0][0]    │
│ (UpSampling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 240, 240,  │          0 │ up_sampling2d_1[… │
│ (Concatenate)       │ 96)               │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 240, 240,  │     27,680 │ concatenate_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 240, 240,  │      9,248 │ conv2d_8[0][0]  

 Total params: 471,907 (1.80 MB)

 Trainable params: 471,907 (1.80 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Callbacks

callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=10, min_lr=1e-7, mode='max'),
    tf.keras.callbacks.EarlyStopping(monitor='val_dice_coef', patience=20, restore_best_weights=True, mode='max'),
    tf.keras.callbacks.ModelCheckpoint('best_multi_model.h5', monitor='val_dice_coef', save_best_only=True, mode='max')
]

In [ ]:
# Training

print("Starting training with 3-channel model...")
history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=100,  
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    shuffle=True,
    verbose=1
)

Starting training with 3-channel model...
Epoch 1/100


I0000 00:00:1767159513.271930      85 service.cc:148] XLA service 0x7e5740004ab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767159513.272632      85 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1767159513.964116      85 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/205 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - accuracy: 0.3539 - dice_coef: 0.0061 - loss: 1.6875

I0000 00:00:1767159521.401809      85 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


205/205 ━━━━━━━━━━━━━━━━━━━━ 28s 75ms/step - accuracy: 0.0197 - dice_coef: 0.0255 - loss: 1.3120 - val_accuracy: 0.0039 - val_dice_coef: 0.3631 - val_loss: 0.6521 - learning_rate: 1.0000e-04
Epoch 2/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.0041 - dice_coef: 0.3801 - loss: 0.6343 - val_accuracy: 0.0036 - val_dice_coef: 0.3801 - val_loss: 0.6301 - learning_rate: 1.0000e-04
Epoch 3/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.0042 - dice_coef: 0.4404 - loss: 0.5703 - val_accuracy: 0.0049 - val_dice_coef: 0.5106 - val_loss: 0.5012 - learning_rate: 1.0000e-04
Epoch 4/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.0054 - dice_coef: 0.5855 - loss: 0.4225 - val_accuracy: 0.0057 - val_dice_coef: 0.5799 - val_loss: 0.4319 - learning_rate: 1.0000e-04
Epoch 5/100
205/205 ━━━━━━━━━━━━━━━━━━━━ 11s 54ms/step - accuracy: 0.0064 - dice_coef: 0.6513 - loss: 0.3573 - val_accuracy: 0.0065 - val_dice_coef: 0.6752 - val_loss: 0.3322 - learning_rate: 1.0000e-

In [ ]:
# Evaluation Matrices

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Enhanced metrics functions
def iou_coef(y_true, y_pred, smooth=1e-6):
    """Intersection over Union (Jaccard Index)"""
    intersection = tf.reduce_sum(tf.abs(y_true * y_pred), axis=[1,2,3])
    union = tf.reduce_sum(y_true, axis=[1,2,3]) + tf.reduce_sum(y_pred, axis=[1,2,3]) - intersection
    iou = tf.reduce_mean((intersection + smooth) / (union + smooth))
    return iou

def precision_metric(y_true, y_pred):
    """Precision metric"""
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

def recall_metric(y_true, y_pred):
    """Recall metric (Sensitivity)"""
    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true * y_pred, 0, 1)))
    possible_positives = tf.reduce_sum(tf.round(tf.clip_by_value(y_true, 0, 1)))
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

def specificity_metric(y_true, y_pred):
    """Specificity metric"""
    true_negatives = tf.reduce_sum(tf.round(tf.clip_by_value((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = tf.reduce_sum(tf.round(tf.clip_by_value(1-y_true, 0, 1)))
    specificity = true_negatives / (possible_negatives + tf.keras.backend.epsilon())
    return specificity

def f1_score(y_true, y_pred):
    """F1-Score metric"""
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

# Enhanced evaluation function
def comprehensive_evaluation(model, X_val, y_val, threshold=0.5):
    """Comprehensive evaluation with medical segmentation metrics"""
    
    print("=" * 70)
    print("COMPREHENSIVE MODEL EVALUATION - MEDICAL SEGMENTATION")
    print("=" * 70)
    
    # Get predictions
    y_pred = model.predict(X_val, verbose=0)
    y_pred_binary = (y_pred > threshold).astype(np.float32)
    
    # Flatten for overall metrics
    y_true_flat = y_val.flatten()
    y_pred_flat = y_pred_binary.flatten()
    
    # Calculate metrics
    val_loss = model.evaluate(X_val, y_val, verbose=0)[0]
    
    # Overall metrics
    overall_dice = dice_coef(
        tf.convert_to_tensor(y_val), 
        tf.convert_to_tensor(y_pred)
    ).numpy()
    
    overall_iou = iou_coef(
        tf.convert_to_tensor(y_val), 
        tf.convert_to_tensor(y_pred_binary)
    ).numpy()
    
    # Confusion matrix
    cm = confusion_matrix(y_true_flat, y_pred_flat)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate rates
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\nOVERALL PERFORMANCE METRICS:")
    print(f"   Loss: {val_loss:.4f}")
    print(f"   Dice Coefficient: {overall_dice:.4f}")
    print(f"   IoU (Jaccard): {overall_iou:.4f}")
    print(f"   F1-Score: {f1:.4f}")
    
    print(f"\nCLASSIFICATION METRICS:")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall (Sensitivity): {recall:.4f}") 
    print(f"   Specificity: {specificity:.4f}")
    
    print(f"\nCONFUSION MATRIX (Pixel-wise):")
    print(f"   True Negatives: {tn:,}")
    print(f"   False Positives: {fp:,}")
    print(f"   False Negatives: {fn:,}")
    print(f"   True Positives: {tp:,}")

    return {
        'loss': val_loss,
        'dice': overall_dice,
        'iou': overall_iou,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'specificity': specificity
    }


print("PERFORMING COMPREHENSIVE EVALUATION...")

# Load best model with enhanced metrics
print("Loading best model with enhanced metrics...")
best_model = tf.keras.models.load_model('best_multi_model.h5', 
    custom_objects={
        'combined_loss': combined_loss,
        'dice_coef': dice_coef,
        'dice_loss': dice_loss,
        'iou_coef': iou_coef,
        'precision_metric': precision_metric,
        'recall_metric': recall_metric,
        'specificity_metric': specificity_metric,
        'f1_score': f1_score
    }
)

# Run comprehensive evaluation
results = comprehensive_evaluation(best_model, X_val, y_val)

In [ ]:
# ==================== SIMPLE SIDE-BY-SIDE VISUALIZATION ====================

def visualize_comparison(model, X_data, y_true, num_samples=3):
    """Simple side-by-side: MRI vs Ground Truth vs Prediction"""
    
    indices = np.random.choice(len(X_data), num_samples, replace=False)
    
    for i, idx in enumerate(indices):
        # Get data
        X_sample = X_data[idx]
        y_true_sample = y_true[idx]
        y_pred = model.predict(X_sample[np.newaxis, ...], verbose=0)[0]
        y_pred_binary = (y_pred > 0.5).astype(np.float32)
        
        # Use FLAIR channel as background
        background = X_sample[:, :, 0]
        
        # Create figure
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # 1. Original MRI
        axes[0].imshow(background, cmap='gray')
        axes[0].set_title('Input MRI')
        axes[0].axis('off')
        
        # 2. Ground Truth (combine all tumor regions)
        axes[1].imshow(background, cmap='gray')
        # Create binary mask of all tumor regions
        gt_combined = np.any(y_true_sample > 0, axis=-1)
        axes[1].imshow(gt_combined, cmap='Reds', alpha=0.5)
        axes[1].set_title('Ground Truth')
        axes[1].axis('off')
        
        # 3. Prediction (combine all tumor regions)
        axes[2].imshow(background, cmap='gray')
        pred_combined = np.any(y_pred_binary > 0, axis=-1)
        axes[2].imshow(pred_combined, cmap='Reds', alpha=0.5)
        
        # Calculate Dice score
        dice = dice_coef(
            tf.convert_to_tensor(y_true_sample[np.newaxis, ...]),
            tf.convert_to_tensor(y_pred[np.newaxis, ...])
        ).numpy()
        
        axes[2].set_title(f'Prediction\nDice: {dice:.3f}')
        axes[2].axis('off')
        
        plt.suptitle(f'Sample {i+1}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

# Execute
print("Generating side-by-side visualizations...")
visualize_comparison(best_model, X_val, y_val, num_samples=3)